# Assignment 4: Reading sensor and publishing data via MQTT Protocol

This notebook demonstrates how to read data from a sensor and publish them via MQTT protocol

Link to the workshop assignment: https://developer.espressif.com/workshops/micropython-jupyter-notebooks-in-browser/assignment-4/

In [ ]:
import time
import struct
import network
import socket
import math
from machine import Pin, I2C
from binascii import hexlify
from micropython import const

try:
    from typing import Tuple
except ImportError:
    pass

## MQTT Client Class

In [ ]:
class MQTTException(Exception):
    pass


class MQTTClient:
    def __init__(
        self,
        client_id,
        server,
        port=0,
        user=None,
        password=None,
        keepalive=0,
        ssl=None,
        ssl_params={},
    ):
        if port == 0:
            port = 8883 if ssl else 1883
        self.client_id = client_id
        self.sock = None
        self.server = server
        self.port = port
        self.ssl = ssl
        self.ssl_params = ssl_params
        self.pid = 0
        self.cb = None
        self.user = user
        self.pswd = password
        self.keepalive = keepalive
        self.lw_topic = None
        self.lw_msg = None
        self.lw_qos = 0
        self.lw_retain = False

    def _send_str(self, s):
        self.sock.write(struct.pack("!H", len(s)))
        self.sock.write(s)

    def _recv_len(self):
        n = 0
        sh = 0
        while 1:
            b = self.sock.read(1)[0]
            n |= (b & 0x7F) << sh
            if not b & 0x80:
                return n
            sh += 7

    def set_callback(self, f):
        self.cb = f

    def set_last_will(self, topic, msg, retain=False, qos=0):
        assert 0 <= qos <= 2
        assert topic
        self.lw_topic = topic
        self.lw_msg = msg
        self.lw_qos = qos
        self.lw_retain = retain

    def connect(self, clean_session=True, timeout=None):
        self.sock = socket.socket()
        self.sock.settimeout(timeout)
        addr = socket.getaddrinfo(self.server, self.port)[0][-1]
        self.sock.connect(addr)
        if self.ssl is True:
            # Legacy support for ssl=True and ssl_params arguments.
            import ssl

            self.sock = ssl.wrap_socket(self.sock, **self.ssl_params)
        elif self.ssl:
            self.sock = self.ssl.wrap_socket(self.sock, server_hostname=self.server)
        premsg = bytearray(b"\x10\0\0\0\0\0")
        msg = bytearray(b"\x04MQTT\x04\x02\0\0")

        sz = 10 + 2 + len(self.client_id)
        msg[6] = clean_session << 1
        if self.user:
            sz += 2 + len(self.user) + 2 + len(self.pswd)
            msg[6] |= 0xC0
        if self.keepalive:
            assert self.keepalive < 65536
            msg[7] |= self.keepalive >> 8
            msg[8] |= self.keepalive & 0x00FF
        if self.lw_topic:
            sz += 2 + len(self.lw_topic) + 2 + len(self.lw_msg)
            msg[6] |= 0x4 | (self.lw_qos & 0x1) << 3 | (self.lw_qos & 0x2) << 3
            msg[6] |= self.lw_retain << 5

        i = 1
        while sz > 0x7F:
            premsg[i] = (sz & 0x7F) | 0x80
            sz >>= 7
            i += 1
        premsg[i] = sz

        self.sock.write(premsg, i + 2)
        self.sock.write(msg)
        self._send_str(self.client_id)
        if self.lw_topic:
            self._send_str(self.lw_topic)
            self._send_str(self.lw_msg)
        if self.user:
            self._send_str(self.user)
            self._send_str(self.pswd)
        resp = self.sock.read(4)
        assert resp[0] == 0x20 and resp[1] == 0x02
        if resp[3] != 0:
            raise MQTTException(resp[3])
        return resp[2] & 1

    def disconnect(self):
        self.sock.write(b"\xe0\0")
        self.sock.close()

    def ping(self):
        self.sock.write(b"\xc0\0")

    def publish(self, topic, msg, retain=False, qos=0):
        pkt = bytearray(b"\x30\0\0\0")
        pkt[0] |= qos << 1 | retain
        sz = 2 + len(topic) + len(msg)
        if qos > 0:
            sz += 2
        assert sz < 2097152
        i = 1
        while sz > 0x7F:
            pkt[i] = (sz & 0x7F) | 0x80
            sz >>= 7
            i += 1
        pkt[i] = sz
        # print(hex(len(pkt)), hexlify(pkt, ":"))
        self.sock.write(pkt, i + 1)
        self._send_str(topic)
        if qos > 0:
            self.pid += 1
            pid = self.pid
            struct.pack_into("!H", pkt, 0, pid)
            self.sock.write(pkt, 2)
        self.sock.write(msg)
        if qos == 1:
            while 1:
                op = self.wait_msg()
                if op == 0x40:
                    sz = self.sock.read(1)
                    assert sz == b"\x02"
                    rcv_pid = self.sock.read(2)
                    rcv_pid = rcv_pid[0] << 8 | rcv_pid[1]
                    if pid == rcv_pid:
                        return
        elif qos == 2:
            assert 0

    def subscribe(self, topic, qos=0):
        assert self.cb is not None, "Subscribe callback is not set"
        pkt = bytearray(b"\x82\0\0\0")
        self.pid += 1
        struct.pack_into("!BH", pkt, 1, 2 + 2 + len(topic) + 1, self.pid)
        # print(hex(len(pkt)), hexlify(pkt, ":"))
        self.sock.write(pkt)
        self._send_str(topic)
        self.sock.write(qos.to_bytes(1, "little"))
        while 1:
            op = self.wait_msg()
            if op == 0x90:
                resp = self.sock.read(4)
                # print(resp)
                assert resp[1] == pkt[2] and resp[2] == pkt[3]
                if resp[3] == 0x80:
                    raise MQTTException(resp[3])
                return

    # Wait for a single incoming MQTT message and process it.
    # Subscribed messages are delivered to a callback previously
    # set by .set_callback() method. Other (internal) MQTT
    # messages processed internally.
    def wait_msg(self):
        res = self.sock.read(1)
        self.sock.setblocking(True)
        if res is None:
            return None
        if res == b"":
            raise OSError(-1)
        if res == b"\xd0":  # PINGRESP
            sz = self.sock.read(1)[0]
            assert sz == 0
            return None
        op = res[0]
        if op & 0xF0 != 0x30:
            return op
        sz = self._recv_len()
        topic_len = self.sock.read(2)
        topic_len = (topic_len[0] << 8) | topic_len[1]
        topic = self.sock.read(topic_len)
        sz -= topic_len + 2
        if op & 6:
            pid = self.sock.read(2)
            pid = pid[0] << 8 | pid[1]
            sz -= 2
        msg = self.sock.read(sz)
        self.cb(topic, msg)
        if op & 6 == 2:
            pkt = bytearray(b"\x40\x02\0\0")
            struct.pack_into("!H", pkt, 2, pid)
            self.sock.write(pkt)
        elif op & 6 == 4:
            assert 0
        return op

    # Checks whether a pending message from server is available.
    # If not, returns immediately with None. Otherwise, does
    # the same processing as wait_msg.
    def check_msg(self):
        self.sock.setblocking(False)
        return self.wait_msg()

In [ ]:
# WiFi credentials
SSID = "<WIFI_NAME>"
PASSWORD = "<WIFI_PASSWORD>"

# Connect WiFi
sta = network.WLAN(network.STA_IF)
sta.active(True)
sta.connect(SSID, PASSWORD)

while not sta.isconnected():
    time.sleep(1)
    print("Connecting to WiFi...")

print("Connected to WiFi")
print(f"Network config: {sta.ifconfig()}")

In [ ]:
# MQTT broker info
BROKER = "test.mosquitto.org"
TOPIC = b"esp32/orientation"

# Setup MQTT
client = MQTTClient("esp32client", BROKER)
client.connect()
print("Connected to MQTT broker")

## ICM42670 Driver Class

In [ ]:
# Device IDs
ICM42607_ID = 0x60
ICM42670_ID = 0x67

# Register addresses
ICM42670_WHOAMI = 0x75
ICM42670_GYRO_CONFIG0 = 0x20
ICM42670_ACCEL_CONFIG0 = 0x21
ICM42670_TEMP_CONFIG = 0x22
ICM42670_PWR_MGMT0 = 0x1F
ICM42670_TEMP_DATA = 0x09
ICM42670_ACCEL_DATA = 0x0B
ICM42670_GYRO_DATA = 0x11

# MREG registers (for advanced functionality)
ICM42670_BLK_SEL_W = 0x79
ICM42670_MADDR_W = 0x7A
ICM42670_M_W = 0x7B
ICM42670_BLK_SEL_R = 0x7C
ICM42670_MADDR_R = 0x7D
ICM42670_M_R = 0x7E

# Constants
ALPHA = 0.99  # Complementary filter weight
RAD_TO_DEG = 57.27272727

# Gyroscope sensitivities (LSB/dps)
GYRO_SENSITIVITIES = {
    0: 16.4,   # ±2000 dps
    1: 32.8,   # ±1000 dps
    2: 65.5,   # ±500 dps
    3: 131.0   # ±250 dps
}

# Accelerometer sensitivities (LSB/g)
ACCEL_SENSITIVITIES = {
    0: 2048,   # ±16g
    1: 4096,   # ±8g
    2: 8192,   # ±4g
    3: 16384   # ±2g
}

# Enums for configuration
class GyroFS:
    FS_2000DPS = 0
    FS_1000DPS = 1
    FS_500DPS = 2
    FS_250DPS = 3

class AccelFS:
    FS_16G = 0
    FS_8G = 1
    FS_4G = 2
    FS_2G = 3

class ODR:
    ODR_1_5625_HZ = 0x01
    ODR_3_125_HZ = 0x02
    ODR_6_25_HZ = 0x03
    ODR_12_5_HZ = 0x04
    ODR_25_HZ = 0x05
    ODR_50_HZ = 0x06
    ODR_100_HZ = 0x07
    ODR_200_HZ = 0x08
    ODR_400_HZ = 0x09
    ODR_800_HZ = 0x0A
    ODR_1600_HZ = 0x0B

class PowerMode:
    OFF = 0
    STANDBY = 1
    LOW_POWER = 2
    LOW_NOISE = 3

class ICM42670:
    """ICM42670/ICM42607 6-axis IMU driver for MicroPython"""
    
    def __init__(self, i2c, address=0x68):
        """
        Initialize the ICM42670 sensor
        
        Args:
            i2c: I2C bus instance
            address: I2C device address (default 0x68)
        """
        self.i2c = i2c
        self.address = address
        self.counter = 0
        self.last_time = 0
        self.dt = 0
        self.roll = 0
        self.pitch = 0
        
        # Check device presence
        device_id = self.get_device_id()
        if device_id not in [ICM42607_ID, ICM42670_ID]:
            raise ValueError(f"Device not found or incorrect ID: 0x{device_id:02x}")
        
        self.device_name = "ICM42607" if device_id == ICM42607_ID else "ICM42670"
        print(f"Found device: {self.device_name}, ID: 0x{device_id:02x}")
    
    def _write_register(self, register, value):
        """Write a single byte to a register"""
        if isinstance(value, int):
            value = bytes([value])
        self.i2c.writeto_mem(self.address, register, value)
    
    def _read_register(self, register, length=1):
        """Read one or more bytes from a register"""
        return self.i2c.readfrom_mem(self.address, register, length)
    
    def get_device_id(self):
        """Get the device ID"""
        for attempt in range(5):
            try:
                device_id = self._read_register(ICM42670_WHOAMI)[0]
                return device_id
            except:
                time.sleep_ms(10)
        raise RuntimeError("Failed to read device ID after 5 attempts")
    
    def configure(self, gyro_fs=GyroFS.FS_2000DPS, gyro_odr=ODR.ODR_100_HZ,
                  accel_fs=AccelFS.FS_16G, accel_odr=ODR.ODR_100_HZ):
        """
        Configure gyroscope and accelerometer settings
        
        Args:
            gyro_fs: Gyroscope full scale range
            gyro_odr: Gyroscope output data rate
            accel_fs: Accelerometer full scale range
            accel_odr: Accelerometer output data rate
        """
        # Configure gyroscope
        gyro_config = ((gyro_fs & 0x03) << 5) | (gyro_odr & 0x0F)
        self._write_register(ICM42670_GYRO_CONFIG0, gyro_config)
        
        # Configure accelerometer
        accel_config = ((accel_fs & 0x03) << 5) | (accel_odr & 0x0F)
        self._write_register(ICM42670_ACCEL_CONFIG0, accel_config)
    
    def set_accel_power_mode(self, mode):
        """Set accelerometer power mode"""
        current = self._read_register(ICM42670_PWR_MGMT0)[0]
        new_value = (current & 0xFC) | (mode & 0x03)
        self._write_register(ICM42670_PWR_MGMT0, new_value)
    
    def set_gyro_power_mode(self, mode):
        """Set gyroscope power mode"""
        current = self._read_register(ICM42670_PWR_MGMT0)[0]
        new_value = (current & 0xF3) | ((mode & 0x03) << 2)
        self._write_register(ICM42670_PWR_MGMT0, new_value)
    
    def get_accel_sensitivity(self):
        """Get current accelerometer sensitivity"""
        config = self._read_register(ICM42670_ACCEL_CONFIG0)[0]
        fs = (config >> 5) & 0x03
        return ACCEL_SENSITIVITIES[fs]
    
    def get_gyro_sensitivity(self):
        """Get current gyroscope sensitivity"""
        config = self._read_register(ICM42670_GYRO_CONFIG0)[0]
        fs = (config >> 5) & 0x03
        return GYRO_SENSITIVITIES[fs]
    
    def get_temperature_raw(self):
        """Get raw temperature value"""
        data = self._read_register(ICM42670_TEMP_DATA, 2)
        return struct.unpack('>H', data)[0]  # Big-endian unsigned short
    
    def get_temperature(self):
        """Get temperature in Celsius"""
        raw_temp = self.get_temperature_raw()
        return (raw_temp / 128.0) + 25.0
    
    def get_accel_raw(self):
        """Get raw accelerometer values (x, y, z)"""
        data = self._read_register(ICM42670_ACCEL_DATA, 6)
        x, y, z = struct.unpack('>hhh', data)  # Big-endian signed shorts
        return (x, y, z)
    
    def get_gyro_raw(self):
        """Get raw gyroscope values (x, y, z)"""
        data = self._read_register(ICM42670_GYRO_DATA, 6)
        x, y, z = struct.unpack('>hhh', data)  # Big-endian signed shorts
        return (x, y, z)
    
    def get_accel_data(self):
        """Get accelerometer data in g units"""
        x_raw, y_raw, z_raw = self.get_accel_raw()
        sensitivity = self.get_accel_sensitivity()
        
        x = x_raw / sensitivity
        y = y_raw / sensitivity
        z = z_raw / sensitivity
        
        return (x, y, z)
    
    def get_gyro_data(self):
        """Get gyroscope data in degrees per second"""
        x_raw, y_raw, z_raw = self.get_gyro_raw()
        sensitivity = self.get_gyro_sensitivity()
        
        x = x_raw / sensitivity
        y = y_raw / sensitivity
        z = z_raw / sensitivity
        
        return (x, y, z)
    
    def get_all_data(self):
        """Get all sensor data at once"""
        accel = self.get_accel_data()
        gyro = self.get_gyro_data()
        temp = self.get_temperature()
        
        return {
            'accel': {'x': accel[0], 'y': accel[1], 'z': accel[2]},
            'gyro': {'x': gyro[0], 'y': gyro[1], 'z': gyro[2]},
            'temperature': temp
        }
    
    def complementary_filter(self, accel_data, gyro_data):
        """
        Apply complementary filter to get roll and pitch angles
    
        Args:
            accel_data: Tuple of (x, y, z) accelerometer values in g
            gyro_data: Tuple of (x, y, z) gyroscope values in dps
        
        Returns:
            Dictionary with 'roll' and 'pitch' angles in degrees
        """
        def normalize_angle(angle):
            # Wrap angle to (-180, 180]
            while angle <= -180:
                angle += 360
            while angle > 180:
                angle -= 360
            return angle
    
        current_time = time.ticks_ms()
    
        self.counter += 1
    
        if self.counter == 1:
            # Initialize angles from accelerometer
            accel_roll = math.atan2(accel_data[1], math.sqrt(accel_data[0]**2 + accel_data[2]**2)) * RAD_TO_DEG
            accel_pitch = math.atan2(-accel_data[0], math.sqrt(accel_data[1]**2 + accel_data[2]**2)) * RAD_TO_DEG
            self.roll = normalize_angle(accel_roll)
            self.pitch = normalize_angle(accel_pitch)
            self.last_time = current_time
            return {'roll': self.roll, 'pitch': self.pitch}
    
        # Calculate time delta in seconds
        dt_ms = time.ticks_diff(current_time, self.last_time)
        self.dt = dt_ms / 1000.0
        self.last_time = current_time
    
        # Calculate angles from accelerometer
        accel_roll = math.atan2(accel_data[1], math.sqrt(accel_data[0]**2 + accel_data[2]**2)) * RAD_TO_DEG
        accel_pitch = math.atan2(-accel_data[0], math.sqrt(accel_data[1]**2 + accel_data[2]**2)) * RAD_TO_DEG
    
        # Normalize accelerometer angles
        accel_roll = normalize_angle(accel_roll)
        accel_pitch = normalize_angle(accel_pitch)
    
        # Calculate gyroscope angle changes
        gyro_roll_delta = gyro_data[0] * self.dt
        gyro_pitch_delta = gyro_data[1] * self.dt
    
        # Apply complementary filter
        self.roll = (ALPHA * (self.roll + gyro_roll_delta)) + ((1 - ALPHA) * accel_roll)
        self.pitch = (ALPHA * (self.pitch + gyro_pitch_delta)) + ((1 - ALPHA) * accel_pitch)
    
        # CRITICAL: Normalize the filtered angles to prevent accumulation
        self.roll = normalize_angle(self.roll)
        self.pitch = normalize_angle(self.pitch)
    
        return {'roll': self.roll, 'pitch': self.pitch}
    
    def read_mreg_register(self, mreg, reg):
        """
        Read from MREG (Memory Register) - advanced functionality
        
        Args:
            mreg: MREG bank (1, 2, or 3)
            reg: Register address within MREG
            
        Returns:
            Register value
        """
        if mreg == 1:
            blk_sel = 0x00
        elif mreg == 2:
            blk_sel = 0x28
        elif mreg == 3:
            blk_sel = 0x50
        else:
            raise ValueError("Invalid MREG value, must be 1, 2, or 3")
        
        self._write_register(ICM42670_BLK_SEL_R, blk_sel)
        self._write_register(ICM42670_MADDR_R, reg)
        time.sleep_us(10)
        value = self._read_register(ICM42670_M_R)[0]
        time.sleep_us(10)
        
        return value
    
    def write_mreg_register(self, mreg, reg, value):
        """
        Write to MREG (Memory Register) - advanced functionality
        
        Args:
            mreg: MREG bank (1, 2, or 3)
            reg: Register address within MREG
            value: Value to write
        """
        if mreg == 1:
            blk_sel = 0x00
        elif mreg == 2:
            blk_sel = 0x28
        elif mreg == 3:
            blk_sel = 0x50
        else:
            raise ValueError("Invalid MREG value, must be 1, 2, or 3")
        
        self._write_register(ICM42670_BLK_SEL_W, blk_sel)
        self._write_register(ICM42670_MADDR_W, reg)
        self._write_register(ICM42670_M_W, value)
        time.sleep_us(10)

In [ ]:
# Initialize I2C (id is 0, look at the back of the board for scl, sda pins, frequency is 400000)
# https://docs.micropython.org/en/latest/library/machine.I2C.html
i2c = ...

# Create sensor instance
imu = ICM42670(i2c)

# Configure sensor
imu.configure(
    gyro_fs=GyroFS.FS_500DPS,
    gyro_odr=ODR.ODR_100_HZ,
    accel_fs=AccelFS.FS_4G,
    accel_odr=ODR.ODR_100_HZ
)

# Enable sensors
imu.set_accel_power_mode(PowerMode.LOW_NOISE)
imu.set_gyro_power_mode(PowerMode.LOW_NOISE)

In [ ]:
def publish_orientation():
    """
    Publish ESP32 orientation (roll, pitch, and text description)
    You will have to determine the rotation direction based on pitch and roll
    https://inertiallabs.com/precision-pitch-and-roll-measurement-enhancing-accuracy-in-navigation-and-stability-control-system/
    """

    while True:
        # Fetch data
        # Check the ICM42670 class and what methods are available for data fetching
        data = ...
        accel = ...
        gyro = ...

        # Compute orientation
        ...

        # Simple orientation interpretation
        ...

        # Publish message
        ...

# Run the orientation example
publish_orientation()